In [302]:
import pandas as pd
import xlrd
import csv
import codecs
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.stats import pearsonr
from minepy import MINE
from sklearn.feature_selection import VarianceThreshold 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score  
from sklearn import metrics

<br>

# xlsx文件类型转换
将xlsx文件转换成为csv方便读取

In [303]:
def xlsx_to_csv(path,doc_name):
    workbook = xlrd.open_workbook(path)
    table = workbook.sheet_by_index(0)
    with codecs.open(doc_name, 'w', encoding='utf-8') as f:
        write = csv.writer(f)
        for row_num in range(table.nrows):
            row_value = table.row_values(row_num)
            write.writerow(row_value)

if __name__ == '__main__':
    xlsx_to_csv('1.银行客户流失预警/3.1.train_sample.xlsx','train.csv')
    xlsx_to_csv('1.银行客户流失预警/3.2.test_sample.xlsx','test.csv')

<br>
<br>
<br>
<br>

# 读取文件

In [304]:
dataframe1 = pd.read_csv('train.csv',sep=',')
dataframe2 = pd.read_csv('test.csv',sep=',')

#讲训练数据与测试数据连接起来，以便一起进行数据清洗
merge_data=pd.concat([dataframe1,dataframe2])  
merge_data.to_csv('final_data.csv',index=False)    #index=False，写入的时候不写入列的索引序号

dataframe = pd.read_csv('final_data.csv',sep=',')

In [305]:
dataframe.head()

,CUST_ID,OPEN_ORG_NUM,IDF_TYP_CD,GENDER,bad_good,LAST_OPEN_TENURE_DAYS,G_OS_PRCP_SUM,OS_PRCP_SUM_THREE,OS_PRCP_SUM_SIX,guozhai_flag,...,L3_CHANNEL_TXN_DTAIN_AVGCNT,L3_CHANNEL_TXN_DOUTTA_AVGCNT,L6_CHANNEL_TXN_STAIN_AVGAMT,L6_CHANNEL_TXN_SOUTTA_AVGAMT,L6_CHANNEL_TXN_DTAIN_AVGAMT,L6_CHANNEL_TXN_DOUTTA_AVGAMT,L6_CHANNEL_TXN_STAIN_AVGCNT,L6_CHANNEL_TXN_SOUTTA_AVGCNT,L6_CHANNEL_TXN_DTAIN_AVGCNT,L6_CHANNEL_TXN_DOUTTA_AVGCNT
0,1000000024,1101,ZR01,2,0.0,0.0,204750.0,197750.00000,98875.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000000515,602,ZR01,1,0.0,0.0,0.0,0.00000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1000003231,602,ZR01,2,1.0,6.0,0.0,466666.66667,233333.33333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1000003736,602,ZR01,1,0.0,0.0,0.0,0.00000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1000003852,602,ZR01,1,0.0,26.0,0.0,0.00000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [306]:
'''plt.figure(figsize=(16,10))
sns.heatmap(dataframe.corr(), square=True,cmap='RdYlGn')
plt.show()'''

"plt.figure(figsize=(16,10))\nsns.heatmap(dataframe.corr(), square=True,cmap='RdYlGn')\nplt.show()"

<br>
<br>
<br>
<br>

# 数据预处理

### 查看缺失值

列级别的判断，只要该列有为空或者NA的元素，就为True，否则False

In [307]:
for i in dataframe.isnull().any():
    if i == True:
        print('There is null value')

<br>
<br>

### 数据格式转换

找出需要被处理的列

In [308]:
columns_to_treat = []
for i in range(len(dataframe.dtypes)):
    dtype = dataframe.dtypes[i]
    if str(dtype) == 'object':
        columns_to_treat.append(i)
        
dict_objects = dict()
for i in columns_to_treat:
    column = dataframe.iloc[:,i]
    if i not in dict_objects:
        dict_objects[i] = []
    for j in column : 
        if j not in dict_objects[i]:
            dict_objects[i].append(j)

In [309]:
# dict_objects

In [310]:
# 证件类型 IDF_TYP_CD 2: ['ZR01', 'ZR20', 'ZR03', 'ZR09', 'ZR22']
for i in range(len(dataframe[dataframe.keys()[2]])):
    sym = int(dataframe['IDF_TYP_CD'][i].split('ZR')[1])
    dataframe.at[i, 'IDF_TYP_CD'] = sym
dataframe['IDF_TYP_CD'] = dataframe['IDF_TYP_CD'].astype(int)

# 173,174,175,176
# 性别 GENDER 3: ['2', '1', 'X']
# 2,1,999
for i in range(len(dataframe[dataframe.keys()[3]])):
    if dataframe['GENDER'][i] == 'X':
        dataframe.at[i,'GENDER'] = 999
    else:
        dataframe.at[i,'GENDER'] = dataframe['GENDER'][i]
dataframe['GENDER'] = dataframe['GENDER'].astype(int)

        
# C_FUND_FLAG 持有货币型基金标志
for m,i,j,k in zip(range(len(dataframe[dataframe.keys()[173]])),range(len(dataframe[dataframe.keys()[174]])),range(len(dataframe[dataframe.keys()[175]])),range(len(dataframe[dataframe.keys()[176]]))):
    sym1 = dataframe[dataframe.keys()[174]][i]
    sym2 = dataframe[dataframe.keys()[175]][j]
    sym3 = dataframe[dataframe.keys()[176]][k]
    sym4 = dataframe[dataframe.keys()[173]][m]
    if sym4 == 'N':
        dataframe.at[m,dataframe.keys()[173]] = 999
    if sym4 == '1':
        dataframe.at[m,dataframe.keys()[173]] = 1
    if sym4 == '0':
        dataframe.at[m,dataframe.keys()[173]] = 0
        
    if sym1 == 'N':
        dataframe.at[i,dataframe.keys()[174]] = 999
    if sym1 == '1':
        dataframe.at[i,dataframe.keys()[174]] = 1
    if sym1 == '0':
        dataframe.at[i,dataframe.keys()[174]] = 0
        
    if sym2 == 'N':
        dataframe.at[j,dataframe.keys()[175]] = 999
    if sym2 == '1':
        dataframe.at[j,dataframe.keys()[175]] = 1
    if sym2 == '0':
        dataframe.at[j,dataframe.keys()[175]] = 0
        
    if sym3 == 'N':
        dataframe.at[k,dataframe.keys()[176]] = 999
    if sym3 == '1':
        dataframe.at[k,dataframe.keys()[176]] = 1
    if sym3 == '0':
        dataframe.at[k,dataframe.keys()[176]] = 0
dataframe[dataframe.keys()[173]] = dataframe[dataframe.keys()[173]].astype(int)
dataframe[dataframe.keys()[174]] = dataframe[dataframe.keys()[173]].astype(int)
dataframe[dataframe.keys()[175]] = dataframe[dataframe.keys()[173]].astype(int)
dataframe[dataframe.keys()[176]] = dataframe[dataframe.keys()[173]].astype(int)


# 所有只关于Y和N的列      
lst = []
for i in dict_objects:
    for j in dict_objects[i]:
        if j != 'N' and j != 'Y':
            break
        else:
            lst.append(i)
for i in lst:
    for j in range(len(dataframe)):
        if dataframe.iloc[j,i] == 'N':
            dataframe.at[j,dataframe.keys()[i]] = 0
        else:
            dataframe.at[j,dataframe.keys()[i]] = 1
    dataframe[dataframe.keys()[i]] = dataframe[dataframe.keys()[i]].astype(int)


In [311]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 627 entries, CUST_ID to L6_CHANNEL_TXN_DOUTTA_AVGCNT
dtypes: float64(593), int64(34)
memory usage: 9.6 MB


<br>
<br>

### 特征选择

<strong>特征是否发散：</strong>如果一个特征不发散，例如方差接近于0，也就是说样本在这个特征上基本上没有差异，这个特征对于样本的区分并没有什么用。此处用的是标准差。<br>
去除标准差为0的列：该列中只有一个值，因此对预测无用<br>

In [312]:
X = dataframe[dataframe.columns.difference(['bad_good'])]
y = dataframe['bad_good']
sel = VarianceThreshold(threshold=(.9 * (1 - .9))) 
new_X = sel.fit_transform(X,y)

# 找出需要删除的变量名
to_drop = []
for i in range(len(sel.get_support())):
    if sel.get_support()[i] == False:
        to_drop.append(X.keys()[i])

删除要删除的变量名字

In [313]:
dataframe = dataframe.drop(to_drop,axis=1)

<br>

#### 卡方检验

查看有无负数

In [314]:
'''columns_with_negatives = []
for i in dataframe.keys():
    lst = list(dataframe[i])
    for j in lst:
        if j < 0:
            columns_with_negatives.append(i)
            break
columns_with_negatives'''

'columns_with_negatives = []\nfor i in dataframe.keys():\n    lst = list(dataframe[i])\n    for j in lst:\n        if j < 0:\n            columns_with_negatives.append(i)\n            break\ncolumns_with_negatives'

对包含负值的列建立新的列，若为负值，则对应位置为1，原来的负值取绝对值

In [315]:
'''for i in columns_with_negatives:
    dataframe[str(i)+'_SYMBOL'] = 0
    
for i in columns_with_negatives:
    for j in range(len(dataframe[i])):
        if dataframe[i][j] < 0:
            dataframe[str(i)+'_SYMBOL'] = 1
            dataframe.at[j,i] = abs(dataframe[i][j])'''

"for i in columns_with_negatives:\n    dataframe[str(i)+'_SYMBOL'] = 0\n    \nfor i in columns_with_negatives:\n    for j in range(len(dataframe[i])):\n        if dataframe[i][j] < 0:\n            dataframe[str(i)+'_SYMBOL'] = 1\n            dataframe.at[j,i] = abs(dataframe[i][j])"

In [316]:
'''from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

X = dataframe[dataframe.columns.difference(['bad_good'])]
y = dataframe['bad_good']
X = SelectKBest(chi2, k=int(len(dataframe.keys())*0.9)).fit_transform(X, y)'''

"from sklearn.feature_selection import SelectKBest\nfrom sklearn.feature_selection import chi2\n\nX = dataframe[dataframe.columns.difference(['bad_good'])]\ny = dataframe['bad_good']\nX = SelectKBest(chi2, k=int(len(dataframe.keys())*0.9)).fit_transform(X, y)"

<br>

#### 互信息数

In [317]:
X = dataframe[dataframe.columns.difference(['bad_good'])]
y = dataframe['bad_good']

to_drop = []
m = MINE()
for i in X.keys():
    m.compute_score(X[i], y)
    if (m.mic()) < 0.005:
        to_drop.append(i)

In [318]:
X = X.drop(to_drop,axis=1)

In [319]:
X.shape

(2000, 343)

<br>

树算法计算特征的信息量：

In [320]:
from sklearn.ensemble import ExtraTreesClassifier 
clf = ExtraTreesClassifier() 
clf.fit(X,y)

# 各个特征重要性 
#print(clf.feature_importances_)
to_drop = []
for i in range(len(clf.feature_importances_)):
    if clf.feature_importances_[i] == 0.0:
        to_drop.append(X.keys()[i])
X = X.drop(to_drop,axis=1)

<br>

KNN，决策树，naive bayes，逻辑回归，SVM，adaboost <br>
https://blog.csdn.net/wu18663419760/article/details/70140009

In [321]:
clf1 = LogisticRegression()  
clf2 = GaussianNB()
clf3 = KNeighborsClassifier()
clf4 = DecisionTreeClassifier()
clf5 = SVC()
clf6 = RandomForestClassifier()

# X:features  y:targets  cv:k  
print(str("LogisticRegression : ") + str(np.mean(cross_val_score(clf1, X, y, cv=5))))
print(str("naive_bayes : ") + str(np.mean(cross_val_score(clf2, X, y, cv=5))))
print(str("KNeighborsClassifier : ") + str(np.mean(cross_val_score(clf3, X, y, cv=5))))
print(str("DecisionTreeClassifier : ") + str(np.mean(cross_val_score(clf4, X, y, cv=5))))
print(str("SVC : ") + str(np.mean(cross_val_score(clf5, X, y, cv=5))))
print(str("RandomForestClassifier : ") + str(np.mean(cross_val_score(clf6, X, y, cv=5))))

LogisticRegression : 0.9099958093488084
naive_bayes : 0.4118339864624154
KNeighborsClassifier : 0.9164970812317577
DecisionTreeClassifier : 0.8659955905974412
SVC : 0.9235021187632423
RandomForestClassifier : 0.9145083219270121


淘汰naive_bayes和DecisionTreeClassifier

<br>

<br>

#### AUC值为ROC曲线所覆盖的区域面积，显然，AUC越大，分类器分类效果越好。

In [125]:
'''from sklearn.cross_validation import cross_val_score, ShuffleSplit
names = dataframe.keys()'''

In [127]:
'''rf = LogisticRegression()
scores = []
for i in range(X.shape[1]):
    score = cross_val_score(rf, X.iloc[:, i:i+1], y, scoring="roc_auc",cv=ShuffleSplit(len(X), 3, .3))
    scores.append((round(np.mean(score), 3), names[i]))
res_LogisticRegression = sorted(scores, reverse=True)'''

In [128]:
'''rf = KNeighborsClassifier()
scores = []
for i in range(X.shape[1]):
    score = cross_val_score(rf, X.iloc[:, i:i+1], y, scoring="roc_auc",cv=ShuffleSplit(len(X), 3, .3))
    scores.append((round(np.mean(score), 3), names[i]))
res_KNeighborsClassifier = sorted(scores, reverse=True)'''

In [130]:
'''rf = DecisionTreeClassifier()
scores = []
for i in range(X.shape[1]):
    score = cross_val_score(rf, X.iloc[:, i:i+1], y, scoring="roc_auc",cv=ShuffleSplit(len(X), 3, .3))
    scores.append((round(np.mean(score), 3), names[i]))
res_DecisionTreeClassifier = sorted(scores, reverse=True)'''

In [131]:
'''rf = SVC()
scores = []
for i in range(X.shape[1]):
    score = cross_val_score(rf, X.iloc[:, i:i+1], y, scoring="roc_auc",cv=ShuffleSplit(len(X), 3, .3))
    scores.append((round(np.mean(score), 3), names[i]))
res_SVC = sorted(scores, reverse=True)'''

In [132]:
'''rf = RandomForestClassifier()
scores = []
for i in range(X.shape[1]):
    score = cross_val_score(rf, X.iloc[:, i:i+1], y, scoring="roc_auc",cv=ShuffleSplit(len(X), 3, .3))
    scores.append((round(np.mean(score), 3), names[i]))
res_RandomForestClassifier = sorted(scores, reverse=True)'''

In [151]:
'''length = int(len(res_LogisticRegression) * 0.9)

low_LogisticRegression = [res_LogisticRegression[i] for i in range(length,len(res_LogisticRegression))]
low_KNeighborsClassifier = [res_KNeighborsClassifier[i] for i in range(length,len(res_KNeighborsClassifier))]
low_DecisionTreeClassifier = [res_DecisionTreeClassifier[i] for i in range(length,len(res_DecisionTreeClassifier))]
low_SVC = [res_SVC[i] for i in range(length,len(res_SVC))]
low_RandomForestClassifier = [res_RandomForestClassifier[i] for i in range(length,len(res_RandomForestClassifier))]
'''

<br>
<br>
<br>
<br>

# 机器学习

### 数据分割

重新拆分为训练集和测试集

In [301]:
dataframe_train = dataframe.loc[0:999]
dataframe_test = dataframe.loc[1000::]

In [322]:
X_train = X[0:999]
y_train = y[0:999]
X_test = X[1000::]
y_test = y[1000::]

<br>

### 数据不均衡处理
https://www.cnblogs.com/xyou/p/9075443.html

In [361]:
cpt1 = 0
for i in dataframe['bad_good']:
    if i == 1.0:
        cpt1+=1
cpt1

153

<strong>conclusion :</strong> target为1的行仅有153行，总行数为999行，数据明显不均衡，需要进行采样处理。

<br>

In [323]:
# 对训练数据集作平衡处理
over_samples = SMOTE(random_state=1234) 
over_samples_X,over_samples_y = over_samples.fit_sample(X_train, y_train)

# 重抽样前的类别比例
print(y_train.value_counts()/len(y_train))
# 重抽样后的类别比例
print(pd.Series(over_samples_y).value_counts()/len(over_samples_y))

0.0    0.914915
1.0    0.085085
Name: bad_good, dtype: float64
1.0    0.5
0.0    0.5
dtype: float64


In [324]:
new_X_train = pd.DataFrame(over_samples_X)
new_y_train = pd.DataFrame(over_samples_y)

修改列名

In [325]:
new_X_train.columns = X.keys()
new_y_train.columns = y.keys()

<br>

### SVC

过采样前

In [337]:
clf1 = SVC()
clf1.fit(X_train, y_train)
y_predict = clf1.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

             precision    recall  f1-score   support

        0.0       0.93      1.00      0.96       932
        1.0       0.00      0.00      0.00        68

avg / total       0.87      0.93      0.90      1000

[[932   0]
 [ 68   0]]


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [338]:
clf1 = SVC()
clf1.fit(new_X_train, new_y_train)
y_predict = clf1.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


             precision    recall  f1-score   support

        0.0       0.93      1.00      0.96       932
        1.0       0.00      0.00      0.00        68

avg / total       0.87      0.93      0.90      1000

[[932   0]
 [ 68   0]]


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [194]:
# RBF 分类模型
# C 惩罚参数C 默认值是1.0 
    # C越大，相当于惩罚松弛变量，希望松弛变量接近0，即对误分类的惩罚增大，趋向于对训练集全分对的情况，这样对训练集测试时准确率很高，
    # 但泛化能力弱。C值小，对误分类的惩罚减小，允许容错，将他们当成噪声点，泛化能力较强。
# gamma ： ‘rbf’,‘poly’ 和‘sigmoid’的核函数参数。默认是’auto’，则会选择1/n_features
clf1

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

调参后 <br>
https://blog.csdn.net/szlcw1/article/details/52336824

无过采样

In [339]:
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
X_train_new = StandardScaler().fit_transform(X_train)

grid = GridSearchCV(SVC(), param_grid={"C":[0.05, 0.1, 0.5, 1, 3, 5], "gamma": [1, 0.1, 0.01]}, cv=3)
grid.fit(X_train_new, y_train)
print(grid.best_params_)
print(grid.best_score_)

{'C': 0.05, 'gamma': 1}
0.914914914914915


In [340]:
clf1 = SVC(C=0.05,gamma=1)
clf1.fit(X_train, y_train)
y_predict = clf1.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

             precision    recall  f1-score   support

        0.0       0.93      1.00      0.96       932
        1.0       0.00      0.00      0.00        68

avg / total       0.87      0.93      0.90      1000

[[932   0]
 [ 68   0]]


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


有过采样

In [348]:
'''import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
X_train_new = StandardScaler().fit_transform(new_X_train)

grid = GridSearchCV(SVC(), param_grid={"C":[0.05, 0.1, 0.5, 1, 3, 5], "gamma": [1, 0.1, 0.01]}, cv=3)
grid.fit(new_X_train, new_y_train)
print(grid.best_params_)
print(grid.best_score_)'''

'import numpy as np\nfrom sklearn.grid_search import GridSearchCV\nfrom sklearn.preprocessing import StandardScaler\nX_train_new = StandardScaler().fit_transform(new_X_train)\n\ngrid = GridSearchCV(SVC(), param_grid={"C":[0.05, 0.1, 0.5, 1, 3, 5], "gamma": [1, 0.1, 0.01]}, cv=3)\ngrid.fit(new_X_train, new_y_train)\nprint(grid.best_params_)\nprint(grid.best_score_)'

<br>

### LogisticRegression

无过采样

In [199]:
clf2 = LogisticRegression()
clf2.fit(X_train, y_train)
y_predict = clf2.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

             precision    recall  f1-score   support

        0.0       0.93      0.98      0.96       932
        1.0       0.17      0.06      0.09        68

avg / total       0.88      0.92      0.90      1000

[[913  19]
 [ 64   4]]


有过采样

In [351]:
clf2 = LogisticRegression()
clf2.fit(new_X_train, new_y_train)
y_predict = clf2.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


             precision    recall  f1-score   support

        0.0       0.95      0.58      0.72       932
        1.0       0.09      0.56      0.15        68

avg / total       0.89      0.58      0.68      1000

[[542 390]
 [ 30  38]]


In [201]:
clf2

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

调参后<br>
https://blog.csdn.net/code_caq/article/details/72027012

In [203]:
new_X_train = StandardScaler().fit_transform(X_train)

grid = GridSearchCV(clf2, param_grid={"C":[0.05, 0.1, 0.5, 1, 3, 5], "solver": ['liblinear', 'lbfgs', 'newton-cg','sag']}, cv=3)
grid.fit(new_X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_

{'C': 0.05, 'solver': 'lbfgs'}
0.8908908908908909


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [214]:
clf2 = LogisticRegression(C=0.05,solver='lbfgs')
clf2.fit(X_train, y_train)
y_predict = clf2.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

             precision    recall  f1-score   support

        0.0       0.93      0.99      0.96       932
        1.0       0.00      0.00      0.00        68

avg / total       0.87      0.92      0.89      1000

[[921  11]
 [ 68   0]]


<br>

### KNeighborsClassifier

无过采样

In [217]:
clf3 = KNeighborsClassifier()
clf3.fit(X_train, y_train)
y_predict = clf3.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

             precision    recall  f1-score   support

        0.0       0.93      0.99      0.96       932
        1.0       0.24      0.06      0.09        68

avg / total       0.89      0.92      0.90      1000

[[919  13]
 [ 64   4]]


有过采样

In [353]:
clf3 = KNeighborsClassifier()
clf3.fit(new_X_train, new_y_train)
y_predict = clf3.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


             precision    recall  f1-score   support

        0.0       0.95      0.78      0.85       932
        1.0       0.12      0.41      0.18        68

avg / total       0.89      0.75      0.81      1000

[[724 208]
 [ 40  28]]


In [218]:
clf3

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

调参后<br>

In [219]:
new_X_train = StandardScaler().fit_transform(X_train)

grid = GridSearchCV(clf3, param_grid={"n_neighbors":[3,5,7,9], 
                                      "leaf_size": [20,25,30,35,40],
                                     "n_jobs":[1,2,3]}, 
                    cv=3)
grid.fit(new_X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)

{'leaf_size': 20, 'n_jobs': 1, 'n_neighbors': 9}
0.8388388388388388


In [221]:
clf3 = KNeighborsClassifier(leaf_size=20,n_neighbors=9)
clf3.fit(X_train, y_train)
y_predict = clf3.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

             precision    recall  f1-score   support

        0.0       0.94      0.99      0.96       932
        1.0       0.33      0.06      0.10        68

avg / total       0.89      0.93      0.90      1000

[[924   8]
 [ 64   4]]


<br>

### DecisionTreeClassifier

无过采样

In [354]:
clf4 = DecisionTreeClassifier()
clf4.fit(X_train, y_train)
y_predict = clf4.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

             precision    recall  f1-score   support

        0.0       0.94      0.91      0.92       932
        1.0       0.12      0.18      0.15        68

avg / total       0.88      0.86      0.87      1000

[[847  85]
 [ 56  12]]


有过采样

In [355]:
clf4 = DecisionTreeClassifier()
clf4.fit(new_X_train, new_y_train)
y_predict = clf4.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

             precision    recall  f1-score   support

        0.0       0.94      0.90      0.92       932
        1.0       0.14      0.22      0.17        68

avg / total       0.89      0.85      0.87      1000

[[840  92]
 [ 53  15]]


In [223]:
clf4

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

调参后<br>
https://blog.csdn.net/akon_wang_hkbu/article/details/77621631

In [231]:
new_X_train = StandardScaler().fit_transform(X_train)

grid = GridSearchCV(clf4, param_grid={"max_features":[80,100,150,200,250], 
                                      "max_depth": [5,10,20,30,40],
                                     }, 
                    cv=3)
grid.fit(new_X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)

{'max_depth': 10, 'max_features': 200}
0.7647647647647647


In [234]:
clf4 = DecisionTreeClassifier(max_depth=10,max_features=200)
clf4.fit(X_train, y_train)
y_predict = clf3.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

             precision    recall  f1-score   support

        0.0       0.94      0.99      0.96       932
        1.0       0.33      0.06      0.10        68

avg / total       0.89      0.93      0.90      1000

[[924   8]
 [ 64   4]]


<br>

### RandomForestClassifier

无过采样

In [356]:
clf5 = RandomForestClassifier()
clf5.fit(X_train, y_train)
y_predict = clf5.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

             precision    recall  f1-score   support

        0.0       0.94      1.00      0.96       932
        1.0       0.50      0.06      0.11        68

avg / total       0.91      0.93      0.91      1000

[[928   4]
 [ 64   4]]


有过采样

In [357]:
clf5 = RandomForestClassifier()
clf5.fit(new_X_train, new_y_train)
y_predict = clf5.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

             precision    recall  f1-score   support

        0.0       0.95      0.93      0.94       932
        1.0       0.23      0.26      0.24        68

avg / total       0.90      0.89      0.89      1000

[[871  61]
 [ 50  18]]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [245]:
clf5

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

调参后<br>
https://blog.csdn.net/yanyanyufei96/article/details/71213351

In [250]:
new_X_train = StandardScaler().fit_transform(X_train)

grid = GridSearchCV(clf5, param_grid={'n_estimators': [10,25,40,55,60],
                                     'max_depth': [2,3,4,5], 
                                     'min_samples_split':[2,4,6,8,10]
                                     },
                    cv=3)
grid.fit(new_X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)

{'max_depth': 2, 'min_samples_split': 2, 'n_estimators': 55}
0.914914914914915


In [251]:
clf5 = RandomForestClassifier(n_estimators = 2, max_depth = 2, min_samples_split = 55)
clf5.fit(X_train, y_train)
y_predict = clf5.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

             precision    recall  f1-score   support

        0.0       0.93      0.99      0.96       932
        1.0       0.30      0.04      0.08        68

avg / total       0.89      0.93      0.90      1000

[[925   7]
 [ 65   3]]


<br>

### xgboost

无过采样

In [255]:
import xgboost
clf6 =  xgboost.XGBClassifier( 
         learning_rate =0.05,
         n_estimators=200,
         max_depth=5,
         min_child_weight=1,
         gamma=0.1,
         subsample=0.8,
         colsample_bytree=0.8,
         objective= 'binary:logistic',
         nthread=4,
         scale_pos_weight=1,
         seed=27)
clf6.fit(X_train,y_train)
res = clf6.predict(X_test)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [256]:
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

             precision    recall  f1-score   support

        0.0       0.93      0.99      0.96       932
        1.0       0.30      0.04      0.08        68

avg / total       0.89      0.93      0.90      1000

[[925   7]
 [ 65   3]]


有过采样

In [358]:
import xgboost
clf6 =  xgboost.XGBClassifier( 
         learning_rate =0.05,
         n_estimators=200,
         max_depth=5,
         min_child_weight=1,
         gamma=0.1,
         subsample=0.8,
         colsample_bytree=0.8,
         objective= 'binary:logistic',
         nthread=4,
         scale_pos_weight=1,
         seed=27)
clf6.fit(new_X_train,new_y_train)
res = clf6.predict(X_test)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [359]:
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

             precision    recall  f1-score   support

        0.0       0.95      0.93      0.94       932
        1.0       0.23      0.26      0.24        68

avg / total       0.90      0.89      0.89      1000

[[871  61]
 [ 50  18]]


In [257]:
clf6

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.1, learning_rate=0.05,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=200, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.8)

调参后<br>

In [ ]:
grid = GridSearchCV(clf6, param_grid={'learning_rate': [0.03,0.05,0.1,0.15,0.2],
                                     'n_estimators': [100,200,300,400,500]
                                     },
                    cv=3)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

In [263]:
clf6 =  xgboost.XGBClassifier( 
         learning_rate =0.15,
         n_estimators=400,
         max_depth=5,
         min_child_weight=1,
         gamma=0.1,
         subsample=0.8,
         colsample_bytree=0.8,
         objective= 'binary:logistic',
         nthread=4,
         scale_pos_weight=1,
         seed=27)
clf6.fit(X_train,y_train)
res = clf6.predict(X_test)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [264]:
print(metrics.classification_report(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))

             precision    recall  f1-score   support

        0.0       0.93      0.99      0.96       932
        1.0       0.30      0.04      0.08        68

avg / total       0.89      0.93      0.90      1000

[[925   7]
 [ 65   3]]
